In [3]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [4]:
import pandas as pd
import numpy as np
from datetime import date

In [5]:
#db.list_libraries()
#db.list_tables('crsp_m_indexes')

In [6]:
info = pd.DataFrame(columns=['Description'])
info.loc['SPX'] = 'S&P500 Index'
info.loc['TB1M'] = 'Treasury Bill (1M) Index'
info

,Description
SPX,S&P500 Index
TB1M,Treasury Bill (1M) Index


In [7]:
sp500 = db.get_table(library='crsp_m_indexes', table='msp500')
sp500.set_index('caldt',inplace=True)
sp500.index = pd.to_datetime(sp500.index)
sp500.index.name = 'date'
sp500 = sp500[['vwretd']]

In [8]:
ytm = db.get_table(library='crsp_m_treasuries', table='tfz_mth_rf')
ytm.set_index('mcaldt',inplace=True)
ytm.index = pd.to_datetime(ytm.index)
ytm.index.name = 'date'
ytm['T-bills'] = ytm['tmytm']/ (100 * 12)

ytm1 = ytm[['T-bills']][ytm['kytreasnox']==2000001.0]
ytm3 = ytm[['T-bills']][ytm['kytreasnox']==2000002.0]

rf = ytm1

In [9]:
df = sp500.join(rf)
df = df.iloc[1:,:]
df.columns = info.index
df

,SPX,TB1M
date,,
1926-01-30,-0.001783,0.003061
1926-02-27,-0.033296,0.002612
1926-03-31,-0.057708,0.002601
1926-04-30,0.038522,0.002270
1926-05-28,0.013623,0.002691
...,...,...
2023-04-28,0.015015,0.003475
2023-05-31,0.005306,0.004129
2023-06-30,0.066952,0.004302


In [10]:
file_out = '../data/barnstable_analysis_data.xlsx'

with pd.ExcelWriter(file_out) as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    df.to_excel(writer, sheet_name='data')